<a href="https://colab.research.google.com/github/nicosesma/otc_strategies/blob/main/Fear_%26_Greed_DCA_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class FearGreedDCAStrategy:
    """
    Implements a DCA strategy based on the Fear & Greed Index.
    """

    def __init__(self,
                 base_order_size: float,
                 fear_threshold: int = 40,
                 extreme_fear_threshold: int = 20,
                 greed_threshold: int = 60,
                 take_profit_percent: float = 5.0):
        """
        Initializes the strategy with core parameters.

        Args:
            base_order_size (float): The size of the initial order in dollars.
            fear_threshold (int): The index value at which to start DCAing.
            extreme_fear_threshold (int): The index value for an aggressive DCA order.
            greed_threshold (int): The index value to trigger a sell/profit-taking.
            take_profit_percent (float): Percentage gain on average price to trigger a sell.
        """
        self.base_order_size = base_order_size
        self.fear_threshold = fear_threshold
        self.extreme_fear_threshold = extreme_fear_threshold
        self.greed_threshold = greed_threshold
        self.take_profit_percent = take_profit_percent / 100.0

    def run_backtest(self, dataframe: pd.DataFrame):
        """
        Simulates the strategy on historical data and provides a detailed report.
        """
        trade_open = False
        trade_info = {}
        trade_history = []

        for i, row in dataframe.iterrows():
            current_price = row['close']
            fg_index = row['fear_greed_index']

            # --- Entry Logic ---
            if not trade_open and fg_index < self.fear_threshold:
                order_size_btc = self.base_order_size / current_price

                trade_info = {
                    'entry_price': current_price,
                    'total_btc_volume': order_size_btc,
                    'total_cost': self.base_order_size,
                    'avg_price': current_price,
                }
                trade_history.append({
                    'Order': 'Base Order (Fear)',
                    'Trigger': f'F&G Index: {fg_index}',
                    'Price': current_price,
                    'Size (BTC)': order_size_btc,
                    'Avg Price': current_price
                })
                trade_open = True

            # --- DCA Logic (on Extreme Fear) ---
            if trade_open and fg_index < self.extreme_fear_threshold:
                # Place an aggressive DCA order
                dca_order_size_usd = self.base_order_size * 1.5  # 50% larger
                dca_order_size_btc = dca_order_size_usd / current_price

                new_total_cost = trade_info['total_cost'] + dca_order_size_usd
                new_total_btc = trade_info['total_btc_volume'] + dca_order_size_btc

                trade_info['total_cost'] = new_total_cost
                trade_info['total_btc_volume'] = new_total_btc
                trade_info['avg_price'] = new_total_cost / new_total_btc

                trade_history.append({
                    'Order': 'DCA Order (Extreme Fear)',
                    'Trigger': f'F&G Index: {fg_index}',
                    'Price': current_price,
                    'Size (BTC)': dca_order_size_btc,
                    'Avg Price': trade_info['avg_price']
                })

            # --- Take Profit Logic ---
            if trade_open:
                take_profit_price = trade_info['avg_price'] * (1 + self.take_profit_percent)
                if current_price >= take_profit_price:
                    pnl = (current_price - trade_info['avg_price']) * trade_info['total_btc_volume']
                    final_roi = (pnl / trade_info['total_cost']) * 100
                    trade_history.append({
                        'Order': 'Take Profit',
                        'Trigger': f'Greed Index > {self.greed_threshold} or TP Hit',
                        'Price': current_price,
                        'PnL ($)': pnl,
                        'ROI (%)': final_roi
                    })
                    trade_open = False
                    break
                elif fg_index > self.greed_threshold:
                    pnl = (current_price - trade_info['avg_price']) * trade_info['total_btc_volume']
                    final_roi = (pnl / trade_info['total_cost']) * 100
                    trade_history.append({
                        'Order': 'Take Profit (Greed)',
                        'Trigger': f'F&G Index: {fg_index}',
                        'Price': current_price,
                        'PnL ($)': pnl,
                        'ROI (%)': final_roi
                    })
                    trade_open = False
                    break

        # If trade is still open at the end
        if trade_open:
            pnl = (current_price - trade_info['avg_price']) * trade_info['total_btc_volume']
            final_roi = (pnl / trade_info['total_cost']) * 100
            trade